In [ ]:
import datasets
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers_interpret import SequenceClassificationExplainer

In [ ]:
##-- Récuperer l'extrait de données np-elec-10k
!wget "https://drive.google.com/uc?export=download&id=17I64kNNUbAyqpEr7lE8eEEfXd7L0NI_r" -O "np-elec-10k.csv"

In [ ]:
##--Importer les données dans un dataframe pandas

df = pd.read_csv("np-elec-10k.csv")  #dispo sur le drive

##--Renommer les colonnes qu'on va utiliser
df = df.rename(columns={ 'Allside': 'labels' })
df = df.rename(columns={ 'highlight.maintext': 'text' })
df = df.rename(columns={ '_source.title': 'title' })
df = df.rename(columns={'_source.source_domain': 'source'})

##--Selectionner les colonnes en question dans un nouveau dataframe
df2 = df[['labels','source','title','text']]

##--Supprimer les labels '?'
df3 = df2[df2['labels'] != '?']

##--Rennommer les labels avec des ints pour faciliter l'utilisation
correspondance = {"Left" :0, "Lean Left" :0, "Center" :1, "Lean Right" :2, "Right" :2, "Mixed" :1}
df3['labels'] = df3['labels'].replace(correspondance)

In [ ]:
 ##-- Trouver les sources uniques dans le DataFrame
sources_uniques = df3['source'].unique()

##- Diviser la liste des sources de manière aléatoire en deux ensembles
sources_train, sources_test = train_test_split(sources_uniques, test_size=0.2, random_state=20)

##-- Filtrer les données en fonction des ensembles de sources
train_df = df3[df3['source'].isin(sources_train)]
test_df = df3[df3['source'].isin(sources_test)]

train_df2 = train_df[['labels','text']]
test_df2 = test_df[['labels','text']]

##--Separer les données dans des fichiers train et test
train_dataset = datasets.Dataset.from_pandas(train_df2)
test_dataset = datasets.Dataset.from_pandas(test_df2)

train_dataset = train_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

##-- Exporter en csv pour verifier le ontenu des datasets
# train.to_csv('./splitm-elec/train.csv')
# test.to_csv('./splitm-elec/test.csv')

dataset = datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000)) #Modifiez la range si vous voulez réduire le temps de training [0:6613]
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) # [0:1202]

## Charger le modèle

In [ ]:
MODEL_SAVE_PATH = ""
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)
model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# Explicabilité
## méthode 1 

In [ ]:
cls_explainer = SequenceClassificationExplainer(model, tokenizer)
cls_explainer(small_eval_dataset[5]["text"][0:511])
cls_explainer.visualize()

## méthode 2

In [ ]:
from torch.nn import Softmax
import json

In [ ]:
results = []

input_text = test_df2.iloc[0]['text']

txt = input_text.split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i in range(832, len(txt)):
  print(f"{i}/{len(txt)}")
  #Partie où on remplace le mots du text par un mask et on prédie avec longformer le mot le plus probable
  cloned_txt = txt[:]
  cloned_txt.pop(i)
  new_txt = ' '.join(cloned_txt)

  #Partie où on va maintenant regarder la classe prédite et la certitude avec le mot altéré

  tokenized_input = tokenizer(new_txt, return_tensors="pt", padding=True, truncation=True).to(device)

  with torch.no_grad():
      logits = model(**tokenized_input).logits

  probabilities = Softmax(dim=1)(logits)
  predicted_class = torch.argmax(logits, dim=1).item()
  confidence = probabilities[0, predicted_class].item()

  #Maitenant on va stocké cette information

  results.append({"predicted_class": predicted_class, "confidence": confidence})

  with open("results.txt", 'w') as fichier:
      # Écrire la valeur de la variable dans le fichier
      fichier.write(json.dumps(results, indent=2))

In [ ]:
with open("results.txt", 'r') as fichier:
    donnees_results = json.load(fichier)

print(donnees_results)

In [ ]:
import matplotlib.pyplot as plt

# Valeur de référence
valeur_reference = 0.9772276282310486

# Extraire les indices, les confiances et les classes pour les 200 premiers éléments
indices = range(len(donnees_results))
confiances = [valeur_reference - element['confidence'] for element in donnees_results]
classes = [element['predicted_class'] for element in donnees_results]

# Créer un graphique à barres
fig, ax = plt.subplots(figsize=(10, 5))
barres = ax.bar(indices, confiances, color=['red' if classe == 0 else 'blue' for classe in classes])

# Ajouter des étiquettes
ax.set_xlabel('Indice')
ax.set_ylabel('Écart par rapport à la valeur de référence')
ax.set_title('Écart par rapport à la valeur de référence')
ax.set_ylim(-0.02, 0.02)
# Afficher le graphique
plt.tight_layout()
plt.show()